# OpenAI API Embedding

In [ ]:
import pandas as pd
import numpy as np
from openai import OpenAI
import tiktoken
from tqdm.notebook import tqdm

# OpenAI 초기화
client = OpenAI(api_key="API키는 저에게 문의하세용용")
enc = tiktoken.encoding_for_model("text-embedding-3-small")

# 원본 데이터 로드
df = pd.read_pickle("/content/shower_processed.pkl")   # tagged_content_text, tagged_comments_text 포함된 파일

# 긴 문서 자르는 함수
def chunk_text(words, max_tokens=2000):
    chunks = []
    for i in range(0, len(words), max_tokens):
        chunks.append(" ".join(words[i:i+max_tokens]))
    return chunks

def embed_text(words):
    text = " ".join(words)
    token_len = len(enc.encode(text))

    if token_len <= 8000:
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=text
        )
        return response.data[0].embedding
    else:
        chunks = chunk_text(words, max_tokens=2000)
        chunk_vecs = []
        for chunk in chunks:
            resp = client.embeddings.create(
                model="text-embedding-3-small",
                input=chunk
            )
            chunk_vecs.append(resp.data[0].embedding)
        return np.mean(chunk_vecs, axis=0).tolist()

# 게시글 임베딩
df["content_vector"] = [embed_text(words) for words in tqdm(df["re_content"])]

# 댓글 임베딩
df["comment_vector"] = [embed_text(words) for words in tqdm(df["re_comment"])]

# 저장
df.to_pickle("with_embeddings_split.pkl")
df.to_csv("with_embeddings_split.csv", index=False)

print("✅ 본문/댓글 분리 임베딩 완료!")


# BERT

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
import pickle  # 🔹 KMeans 모델 저장용

# --- 1. 벡터화 (의미 추출) ---
df = pd.read_pickle("this.pkl")
model = SentenceTransformer("snunlp/KR-SBERT-V40K-klueNLI-augSTS")
sentences = df['text'].tolist()

print("--- (1/3) 문장 벡터화(Encoding) 시작 ---")
embeddings = model.encode(sentences, show_progress_bar=True)
print(f"임베딩 벡터의 크기: {embeddings.shape}")
print("-" * 30)

# --- 2. 최적의 K 찾기 (실루엣 스코어) ---
silhouette_scores = []
k_range = range(2, 11)

print("--- (2/3) 최적의 K 찾는 중 (실루엣 스코어) ---")

for k in tqdm(k_range, desc="K-Means 반복"):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(embeddings)
    labels = kmeans.labels_

    score = silhouette_score(embeddings, labels)
    silhouette_scores.append(score)

print("K값별 실루엣 스코어:", silhouette_scores)
print("-" * 30)

# --- 3. 시각화로 K값 결정 ---
print("--- (3/3) 시각화 및 최종 클러스터링 ---")
plt.figure(figsize=(10, 6))
plt.plot(k_range, silhouette_scores, marker='o')
plt.title('Silhouette Score for Optimal k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Silhouette Score')
plt.xticks(k_range)
plt.grid(True)
plt.show()

optimal_k = k_range[silhouette_scores.index(max(silhouette_scores))]
print(f"그래프 확인 결과, 최적의 K는 {optimal_k} 입니다.")
print("-" * 30)

# --- 4. 최종 클러스터링 실행 ---
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
kmeans.fit(embeddings)
clusters = kmeans.labels_
df['cluster_label'] = clusters

print("--- 최종 클러스터링 결과 (DataFrame) ---")
print(df.sort_values(by='cluster_label'))

# --- 5. 결과 저장 ---
print("--- (4/4) 결과 저장 중 ---")

# (1) 임베딩 벡터 저장
np.save("embeddings.npy", embeddings)

# (2) 클러스터 라벨이 포함된 DataFrame 저장
df.to_pickle("clustered_result.pkl")

# (3) 학습된 KMeans 모델 저장
with open("kmeans_model.pkl", "wb") as f:
    pickle.dump(kmeans, f)

print("✅ 모든 결과 저장 완료!")
print(" - embeddings.npy")
print(" - clustered_result.pkl")
print(" - kmeans_model.pkl")
